In [1]:
import psycopg2
from psycopg2.extensions import register_adapter, AsIs

In [2]:
def insertRow(row,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    insert_statement = 'INSERT into heartbeatstemporal (%s) values %s'
    columns = row.keys()
    values = [row[column] for column in columns]
    cur.execute(insert_statement, (AsIs(','.join(columns)), tuple(values)))
    conn.commit()
    conn.close()

In [3]:
def selectrow(record,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = ("SELECT a.record, a.sample as s_from, b.Sample - 1 as s_to "
                      " FROM R a JOIN R b on a.id = b.id -1 and a.record = b.record"
                      " AND a.record='"+record+"'")
    cur.execute(select_stament)
    for row in cur :
        register = {'record':row[0],'s_from':row[1],'s_to':row[2]}
        insertRow(register,dbname="mimic")

In [4]:
def distinctRecord(dbname="mimic"):
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = "SELECT DISTINCT r.record FROM r WHERE r.record NOT IN (SELECT distinct hb.record FROM heartbeatstemporal hb)"
    cur.execute(select_stament)
    for record in cur :
        print(record[0])
        selectrow(record[0])
    conn.close()

In [5]:
distinctRecord()

mimic2wdb/matched/s04331/s04331-2696-12-22-07-41
mimic2wdb/matched/s14266/s14266-2770-01-13-15-29
mimic2wdb/matched/s14947/s14947-3433-11-06-12-07
mimic2wdb/matched/s15687/s15687-3453%10-21-19-11
